In [1]:
import argparse
import os
import math
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from dataclasses import dataclass
from torch.nn import LayerNorm
from torch.utils.data import dataset, dataloader
print('all the nuccesary libraries and functions are imported')

all the nuccesary libraries and functions are imported


In [2]:
# set different hiperparameters

class HParams:
    block_size: int = 128
    n_layers: int = 6
    n_heads: int = 8
    d_model: int = 512
    d_ff: int= 2048
    dropout: float =0.1
    emb_dropout: float = 0.0
    lr: float = 3e-4
    weight_decay: float = 1e-1
    betas: tuple = (0.9, 0.95)
    batch_size: int = 64
    max_iters: int = 10000
    eval_interval: int = 500
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    seed: int = 42
    grad_clip: float = 1.0
    top_k: int = 40
    temperature: float = 1.0

In [3]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.GELU(),
            nn.Linear(d_ff, d_model),
            nn.Dropout(dropout),
        )
    def forward(self, x):
        return self.net(x)


In [1]:
def build_vocab(text):
    chars = sorted(list(set(text)))
    stoi = {ch:i for i,ch in enumerate(chars)}
    itos = {i:ch for ch,i in stoi.items()}
    return stoi, itos


In [ ]:
class CausalSelfAttention(nn.Module):
    def __init__(self, d_model, n_heads, block_size, dropout):
        super().__init__()
        assert d_model % n_heads == 0
        self.n_heads = n_heads
        self.head_dim = d_model // n_heads
        self.scale = self.head_dim ** -0.5
        self.qkv = nn.Linear(d_model, 3 * d_model)
        self.proj = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)

        # causal mask will be created on the fly in forward (efficient for different devices)
        self.register_buffer("mask", torch.tril(torch.ones(block_size, block_size)).unsqueeze(0).unsqueeze(0))
